In [1]:
import os

In [2]:
%pwd

'e:\\MLops_project\\project-dl-end-to-end-main\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\MLops_project\\project-dl-end-to-end-main'

In [6]:
# mlflow >> to track experiments

In [8]:
#  https://dagshub.com/adiphalke572/my-first-repo.mlflow

In [9]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/adiphalke572/my-first-repo.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="adiphalke572"
os.environ["MLFLOW_TRACKING_PASSWORD"]="6246c68cb22e8410de95bb97fd9851d60198fab8"

In [ ]:
# !pip install mlflow

  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached alembic-1.15.2-py3-none-any.whl.metadata (7.3 kB)
  Using cached matplotlib-3.10.1-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached sqlalchemy-2.0.40-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached sqlparse-0.5.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached contourpy-1.3.1-cp312-cp312-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-

In [11]:
import mlflow
import os

# set mlflow tracking URL
mlflow.set_tracking_uri("https://dagshub.com/adiphalke572/my-first-repo.mlflow")

# set your DagsHub credentials (MLflow uses HTTP Basic Auth)
os.environ["MLFLOW_TRACKING_USERNAME"]="adiphalke572"
os.environ["MLFLOW_TRACKING_PASSWORD"]="6246c68cb22e8410de95bb97fd9851d60198fab8"

In [14]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [15]:
# entity

In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [17]:
# configuration Manager

In [18]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.comman import read_yaml,create_directories,save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/adiphalke572/my-first-repo.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [19]:
# components

In [20]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [21]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [22]:
# pipeline

In [23]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 139 images belonging to 2 classes.


c:\Users\ADITYA\anaconda3\envs\dlenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.4873 - loss: 0.8917


2025/04/14 14:50:45 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/04/14 14:51:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'VGG16Model'.
2025/04/14 14:52:36 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


🏃 View run adaptable-quail-623 at: https://dagshub.com/adiphalke572/my-first-repo.mlflow/#/experiments/0/runs/e04e1ce49a374d6d8305215ad0c98d5c
🧪 View experiment at: https://dagshub.com/adiphalke572/my-first-repo.mlflow/#/experiments/0
